### 형태소 분석 / 불용어 사전 적용 / 감성 사전 적용

### library load
* KoNLP 가 설치되었다고 가정하겠습니다.
* 설치 중에 rJava 관련 에러가 발생하면 자바 JRE/JDK를 다운 받아 설치한 후에 다시 진행합니다.

In [73]:

library(tidytext)
library(tidyverse)
library(KoNLP)

# library(presidentSpeech)

# library(RcppMeCab)
# library(broom)

### 분석하고자 하는 데이터를 가져옵니다.
* 언론재단에서 운영하는 bigkinds 사이트에서 특정 키워드로 다운 받은 데이터입니다.
* 키워드 : 사설 + keywords

In [2]:
data_economy <- read_delim("data/txt_extract/경제.txt", delim="\t")
data_sewol <- read_delim("data/txt_extract/세월호.txt", delim="\t")
data_pay <- read_delim("data/txt_extract/최저임금.txt", delim="\t")
data_demo1 <- read_delim("data/txt_extract/민주당.txt", delim="\t")
data_demo2 <- read_delim("data/txt_extract/더불어민주당.txt", delim="\t")
data_libe1 <- read_delim("data/txt_extract/한국당.txt", delim="\t")
data_libe2 <- read_delim("data/txt_extract/자유한국당.txt", delim="\t")


Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(
  식별자 = col_character(),
  일자 = col_double(),
  언론사 = col_character(),
  제목 = col_character(),
  본문 = col_character()
)
Parsed with column specification:
cols(


* 각각의 데이터를 하나로 합쳤습니다.
* 데이터를 합친 후에 원래의 데이터는 rm(data_economy) 등으로 삭제했습니다.

In [3]:
mutate_01 <- function(df, keyword) {
    
    df  %>% mutate(검색어 = keyword) -> df
    return(df)
}

# mutate_01(data_economy, "경제")
# mutate_01(data_sewol, "세월호")
# mutate_01(data_pay, "최저임금")

bind_rows(mutate_01(data_economy, "경제"),
          mutate_01(data_sewol, "세월호"),
          mutate_01(data_pay, "최저임금"),
          mutate_01(data_demo1, "민주당"),
          mutate_01(data_demo2, "더불어민주당"),
          mutate_01(data_libe1, "한국당"),
          mutate_01(data_libe2, "자유한국당")
          ) -> data



* 분석의 편의를 위해서 컬러명을 영문으로 변경했습니다.

In [14]:
(
    data
     %>% rename(pid = 식별자, date = 일자, news=언론사, title = 제목, article = 본문, keyword = 검색어)
) -> data

* 분석을 위해 row_number를 아래와 같이 부여했습니다. 언론재단에서 제공하는 ID가 너무 길어서 대체했습니다.

In [15]:
(
    data
     %>% mutate(id = row_number())
     %>% select(id,date,news,title, article, keyword)
#      %>% tail(5)
)-> data

* 이제 아래와 같이 분석 대상이 되는 데이터를 만들었습니다.

In [17]:
(
    data
     %>% head(1)
)

id,date,news,title,article,keyword
1,20190613,강원일보,"[사설]내년 총선, 경제 살리기 일자리 대책이 승부 갈라","정계개편이 10개월 앞으로 다가온 내년 4 15 총선의 최대 변수로 부상하고 있다. 우선 `보수 대통합'을 향해 보수진영이 꿈틀대고 있다. 현재 보수는 자유한국당과 옛 새누리당에 뿌리를 둔 바른정당계 바른미래당 의원 일부, 대한애국당 등으로 흩어져 있다. 내년 총선 전 보수진영의 통합이 이뤄질 경우 정계개편은 물론 중도층의 표심에도 적지 않은 파장을 불..",경제


* 데이터가 어떤 것인지 간단하게 살펴보겠습니다.

In [26]:
(
    data
#      %>% distinct(keyword) # 검색대상으로 입력한 키워드입니다. 총7개입니다.
#      %>% distinct(news) # 포함되어 있는 신문사입니다.
     %>% count() # 전체 데이터 갯수는 10,873개 입니다.
#      %>% str()
)

n
10873


* 간단한 분포를 살펴보겠습니다.

In [32]:
(
    data
#      %>% count(news, sort=T) # 신문사별 기사 갯수입니다.
     %>% count(keyword, sort=T) # 검색 키워드별 기사 갯수입니다.
#      %>% summarise()
)

keyword,n
경제,7623
민주당,1637
세월호,631
한국당,504
최저임금,336
자유한국당,128
더불어민주당,14


* 한국당/자유한국당, 민주당/더블어민주당은 키워드 검색 시에 각각 입력한 것입니다. 
* 분석을 위해 merge 하려고 합니다.

In [36]:
data$keyword2 <- str_replace(data$keyword, "자유한국당", "한국당")

In [39]:
(
    data
     %>% mutate(keyword3 = str_replace(data$keyword2, "더불어민주당", "민주당"))
) -> data

In [44]:
(
    data
     %>% select(-keyword, -keyword2, keyword = keyword3)
#      %>% head(5)
) -> data

* 이제 키워드별 통계를 다시 산출해보겠습니다.

In [45]:
(
    data
#      %>% count(news, sort=T) # 신문사별 기사 갯수입니다.
     %>% count(keyword, sort=T) # 검색 키워드별 기사 갯수입니다.
#      %>% summarise()
)

keyword,n
경제,7623
민주당,1651
한국당,632
세월호,631
최저임금,336


* 시계열 분석을 위해 날짜를 연/월/일로 분해해보겠습니다.

In [57]:
(
    data
     %>% mutate(year = substr(date,1,4), 
                month = substr(date,5,6),
               day = substr(date,7,8))
     %>% select(id, date,year, month, day, news, keyword, title, article)
#      %>% head(5)
) -> data

In [95]:
data  %>% head(1)

id,date,year,month,day,news,keyword,title,article
1,20190613,2019,06,13,강원일보,경제,"[사설]내년 총선, 경제 살리기 일자리 대책이 승부 갈라","정계개편이 10개월 앞으로 다가온 내년 4 15 총선의 최대 변수로 부상하고 있다. 우선 `보수 대통합'을 향해 보수진영이 꿈틀대고 있다. 현재 보수는 자유한국당과 옛 새누리당에 뿌리를 둔 바른정당계 바른미래당 의원 일부, 대한애국당 등으로 흩어져 있다. 내년 총선 전 보수진영의 통합이 이뤄질 경우 정계개편은 물론 중도층의 표심에도 적지 않은 파장을 불.."


### KoNLP 기초 함수

* 실습을 위해 데이터에서 일부 데이터만 추출하여 사용하겠습니다. (속도 문제)

In [67]:
(
    data
     %>% filter(news %in% c("조선일보", "경향신문"))
     %>% filter(year %in% c(2019,2018))
#      %>% group_by(news,year, keyword)
#      %>% summarise(CNT = n_distinct(id))
) -> t

In [79]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% unnest_tokens(input = article, output=word, token='words')
#      %>% head(5)
)

id,year,month,news,keyword,word
9,2019,06,조선일보,경제,경제가
9,2019,06,조선일보,경제,탄탄하다
9,2019,06,조선일보,경제,던
9,2019,06,조선일보,경제,정부
9,2019,06,조선일보,경제,여당이
9,2019,06,조선일보,경제,갑자기
9,2019,06,조선일보,경제,대외
9,2019,06,조선일보,경제,여건
9,2019,06,조선일보,경제,악화
9,2019,06,조선일보,경제,를


In [81]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% unnest_tokens(input = article, output=sentence, token='sentences')
#      %>% head(5)
)

id,year,month,news,keyword,sentence
9,2019,06,조선일보,경제,"'경제가 탄탄하다'던 정부 여당이 갑자기 '대외 여건 악화'를 내세우면서 ""추경예산 통과"" 주장만 되풀이하고 있다."
9,2019,06,조선일보,경제,"청와대 경제수석은 '경제 하방 리스크'를 10여 차례나 언급한 뒤 ""추경이 늦어지면 일자리 1만~2만개를 놓칠 수 있다""고 했다."
9,2019,06,조선일보,경제,"마치 추경이 집행되지 않아 경제가 어려움에 처했고, 추경만 통과되면 경제가 살아날 듯 말하고 있다."
9,2019,06,조선일보,경제,처음 보..
13,2019,06,경향신문,경제,"윤종원 청와대 경제수석이 지난 7일 하반기 경제전망에 대해 “한국 경제는 대외여건의 영향을 많이 받는 경제인데, 하방위험이 큰 상황”이라며 “세계 교역량의 가파른 하락과 미 중 간 통상마찰 등으로 하방압력은 장기화할 수 있다”고 말했다."
13,2019,06,경향신문,경제,그는 “경기는 하강 후 바닥을 다지는 국면”이라면서도 “바닥을 다진다는 게 갑자기 한 달 만에 돌아서고 이런 것은 아..
27,2019,06,조선일보,경제,현대중공업이 31일 노조를 피해 울산대 체육관으로 주주총회장을 옮겨서 대우조선해양 합병 절차를 처리했다.
27,2019,06,조선일보,경제,뒤늦게 주총장에 온 민노총 노조원들은 체육관 유리문을 깨고 들어가 소화기액을 뿌리고 의자를 집어던지는 등 난동을 부렸다.
27,2019,06,조선일보,경제,"이재갑 고용노동부 장관은 ""노조 불법행위는 법 절차에 따라 조치하겠다""고 했다."
27,2019,06,조선일보,경제,빈말이란 걸 국민이 다 안다.


In [85]:
extractNoun("이것은 테스트로 해보는 것으로서 이것은 핸드폰입니다.")

[1] "이것"   "테스트" "것"     "이것"   "핸드폰"

In [88]:
SimplePos09("이것은 테스트로 해보는 것으로서 이것은 핸드폰입니다.")

$이것은
[1] "이것/N+은/J"

$테스트로
[1] "테스트/N+로/J"

$해보는
[1] "하/P+어/E+보/P+는/E"

$것으로서
[1] "것/N+으로서/J"

$이것은
[1] "이것/N+은/J"

$핸드폰입니다
[1] "핸드폰/N+이/J+ㅂ니다/E"

$.
[1] "./S"

In [89]:
SimplePos22("이것은 테스트로 해보는 것으로서 이것은 핸드폰입니다.")

$이것은
[1] "이것/NP+은/JX"

$테스트로
[1] "테스트/NC+로/JC"

$해보는
[1] "하/PX+어/EC+보/PX+는/ET"

$것으로서
[1] "것/NB+으로서/JC"

$이것은
[1] "이것/NP+은/JX"

$핸드폰입니다
[1] "핸드폰/NC+이/JP+ㅂ니다/EF"

$.
[1] "./SF"

In [90]:
MorphAnalyzer("이것은 테스트로 해보는 것으로서 이것은 핸드폰입니다.")

$이것은
[1] "이것/npd+은/jxc"

$테스트로
[1] "테스트/ncpa+로/jca"

$해보는
[1] "해보/pvg+는/etm"            "하/pvg+어/ecx+보/px+는/etm"
[3] "하/px+어/ecx+보/px+는/etm" 

$것으로서
[1] "것/nbn+으로서/jca"

$이것은
[1] "이것/npd+은/jxc"

$핸드폰입니다
[1] "핸드폰/ncn+이/jp+ㅂ니다/ef"

$.
[1] "./sf" "./sy"

In [86]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% unnest_tokens(input = article, output=extractnoun, token=extractNoun)
#      %>% head(5)
)

id,year,month,news,keyword,extractnoun
9,2019,06,조선일보,경제,경제
9,2019,06,조선일보,경제,탄탄하다'던
9,2019,06,조선일보,경제,정부
9,2019,06,조선일보,경제,여당
9,2019,06,조선일보,경제,대외
9,2019,06,조선일보,경제,여건
9,2019,06,조선일보,경제,악화
9,2019,06,조선일보,경제,추경예산
9,2019,06,조선일보,경제,통과
9,2019,06,조선일보,경제,주장


In [83]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% unnest_tokens(input = article, output=simplepos09, token=SimplePos09)
#      %>% head(5)
)

id,year,month,news,keyword,simplepos09
9,2019,06,조선일보,경제,'/s+경제/n+가/j
9,2019,06,조선일보,경제,탄탄하다'던/n
9,2019,06,조선일보,경제,정부/n
9,2019,06,조선일보,경제,여당/n+이/j
9,2019,06,조선일보,경제,갑자기/m
9,2019,06,조선일보,경제,'/s+대외/n
9,2019,06,조선일보,경제,여건/n
9,2019,06,조선일보,경제,악화/n+'/s+를/j
9,2019,06,조선일보,경제,내세우/p+면서/e
9,2019,06,조선일보,경제,"""/s+추경예산/n"


In [84]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% unnest_tokens(input = article, output=simplepos22, token=SimplePos22)
#      %>% head(5)
)

id,year,month,news,keyword,simplepos22
9,2019,06,조선일보,경제,'/sr+경제/nc+가/jc
9,2019,06,조선일보,경제,탄탄하다'던/nc
9,2019,06,조선일보,경제,정부/nc
9,2019,06,조선일보,경제,여당/nc+이/jc
9,2019,06,조선일보,경제,갑자기/ma
9,2019,06,조선일보,경제,'/sr+대외/nc
9,2019,06,조선일보,경제,여건/nc
9,2019,06,조선일보,경제,악화/nc+'/sr+를/jc
9,2019,06,조선일보,경제,내세우/pv+면서/ec
9,2019,06,조선일보,경제,"""/sl+추경예산/nc"


In [100]:
(
    t
     %>% select(id, year, month, news, keyword, article)
     %>% filter(year == '2019' & month == '03' & news == '조선일보' & keyword=='경제')
     %>% unnest_tokens(input = article, output=morphanalyzer, token=MorphAnalyzer)
#      %>% head(5)
)

ERROR: Error in `[[<-.data.frame`(`*tmp*`, output, value = structure(c("지나/pvg+ㄴ/etm", : replacement has 1040 rows, data has 323


In [102]:
(
    t
     %>% select(id,title)
     %>% unnest_tokens(input = title,output = m, token=MorphAnalyzer)
)

ERROR: Error in `[[<-.data.frame`(`*tmp*`, output, value = structure(c("[/sl+사설/ncn+]/sr", : replacement has 5022 rows, data has 1781


In [108]:
MorphAnalyzer("롯데마트가 판매하고 있는 흑마늘 양념 치킨이 논란이 되고 있다.")

$롯데마트가
[1] "롯데마트/ncn+가/jcc" "롯데마트/ncn+가/jcs"

$판매하고
[1] "판매/ncpa+하고/jcj"             "판매/ncpa+하고/jct"            
[3] "판매/ncpa+하/xsva+고/ecc"       "판매/ncpa+하/xsva+고/ecs"      
[5] "판매/ncpa+하/xsva+고/ecx"       "판매/ncpa+하/xsva+어/ef+고/jcr"

$있는
[1] "있/paa+는/etm" "있/px+는/etm" 

$흑마늘
[1] "흑마늘/ncn" "흑마늘/nqq"

$양념
[1] "양념/ncn"

$치킨이
[1] "치킨/ncn+이/jcc" "치킨/ncn+이/jcs" "치킨/ncn+이/ncn"

$논란이
[1] "논란/ncpa+이/jcc" "논란/ncpa+이/jcs" "논란/ncpa+이/ncn"

$되고
 [1] "되/nbu+고/jcj"       "되/nbu+이/jp+고/ecc" "되/nbu+이/jp+고/ecs"
 [4] "되/nbu+이/jp+고/ecx" "되/paa+고/ecc"       "되/paa+고/ecs"      
 [7] "되/paa+고/ecx"       "되/pvg+고/ecc"       "되/pvg+고/ecs"      
[10] "되/pvg+고/ecx"       "되/px+고/ecc"        "되/px+고/ecs"       
[13] "되/px+고/ecx"       

$있다
[1] "있/paa+다/ef" "있/px+다/ef" 

$.
[1] "./sf" "./sy"

In [109]:
convertHangulStringToJamos("R는 많은 공헌자에의한 공동 프로젝트입니다")

[1] "R"      "ㄴㅡㄴ" " "      "ㅁㅏㄶ" "ㅇㅡㄴ" " "      "ㄱㅗㅇ" "ㅎㅓㄴ"
 [9] "ㅈㅏ"   "ㅇㅔ"   "ㅇㅢ"   "ㅎㅏㄴ" " "      "ㄱㅗㅇ" "ㄷㅗㅇ" " "     
[17] "ㅍㅡ"   "ㄹㅗ"   "ㅈㅔㄱ" "ㅌㅡ"   "ㅇㅣㅂ" "ㄴㅣ"   "ㄷㅏ"

In [110]:
convertHangulStringToKeyStrokes("R는 많은 공헌자에의한 공동 프로젝트입니다")

[1] "R"        "ｓｍｓ"   " "        "ａｋｓｇ" "ｄｍｓ"   " "       
 [7] "ｒｈｄ"   "ｇｊｓ"   "ｗｋ"     "ｄｐ"     "ｄｍｌ"   "ｇｋｓ"  
[13] " "        "ｒｈｄ"   "ｅｈｄ"   " "        "ｖｍ"     "ｆｈ"    
[19] "ｗｐｒ"   "ｘｍ"     "ｄｌｑ"   "ｓｌ"     "ｅｋ"

In [111]:
str <- convertHangulStringToJamos("배포 조건의 상세한것에 대해서는 'license()' 또는 'licence()' 라고 입력해주십시오")
str2 <-paste(str, collapse="")
str2

[1] "ㅂㅐㅍㅗ ㅈㅗㄱㅓㄴㅇㅢ ㅅㅏㅇㅅㅔㅎㅏㄴㄱㅓㅅㅇㅔ ㄷㅐㅎㅐㅅㅓㄴㅡㄴ 'license()' ㄸㅗㄴㅡㄴ 'licence()' ㄹㅏㄱㅗ ㅇㅣㅂㄹㅕㄱㅎㅐㅈㅜㅅㅣㅂㅅㅣㅇㅗ"

In [112]:
HangulAutomata(str2)

[1] "배포 조건의 상세한것에 대해서는 'license()' 또는 'licence()' 라고 입력해주십시오"

## dic 사용

* SystemDic

In [113]:
useSystemDic()

Backup was just finished!
283949 words dictionary was built.


In [114]:
extractNoun("성긴털제비꽃은 너무 예쁘다.")

[1] "성긴털제비꽃은"

In [115]:
SimplePos09("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃은/N"

$너무
[1] "너무/M"

$예쁘다
[1] "예쁘/P+다/E"

$.
[1] "./S"

In [116]:
SimplePos22("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃은/NC"

$너무
[1] "너무/MA"

$예쁘다
[1] "예쁘/PA+다/EF"

$.
[1] "./SF"

In [117]:
MorphAnalyzer("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/ncn+은/jxc" "성긴털제비꽃은/ncn"     
[3] "성긴털제비꽃/nqq+은/jxc" "성긴털제비꽃은/nqq"     

$너무
[1] "너무/mag"

$예쁘다
[1] "예쁘/paa+다/ef"

$.
[1] "./sf" "./sy"

* SejongDic

In [118]:
useSejongDic()

Backup was just finished!
370957 words dictionary was built.


In [119]:
extractNoun("성긴털제비꽃은 너무 예쁘다.")

[1] "성긴털제비꽃"

In [120]:
SimplePos09("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/N+은/J"

$너무
[1] "너무/M"

$예쁘다
[1] "예쁘/P+다/E"

$.
[1] "./S"

In [121]:
SimplePos22("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/NC+은/JX"

$너무
[1] "너무/MA"

$예쁘다
[1] "예쁘/PA+다/EF"

$.
[1] "./SF"

In [122]:
MorphAnalyzer("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/ncn+은/jxc" "성긴털제비꽃/ncn+은/ncn"

$너무
[1] "너무/mag"

$예쁘다
[1] "예쁘/paa+다/ef"

$.
[1] "./sf" "./sy"

* NIADic()

In [123]:
useNIADic()

Backup was just finished!
983012 words dictionary was built.


In [124]:
extractNoun("성긴털제비꽃은 너무 예쁘다.")

[1] "성긴털" "제비꽃"

In [125]:
SimplePos09("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/N+은/J"

$너무
[1] "너무/M"

$예쁘다
[1] "예쁘/P+다/E"

$.
[1] "./S"

In [126]:
SimplePos22("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털제비꽃/NC+은/JX"

$너무
[1] "너무/MA"

$예쁘다
[1] "예쁘/PA+다/EF"

$.
[1] "./SF"

In [127]:
MorphAnalyzer("성긴털제비꽃은 너무 예쁘다.")

$성긴털제비꽃은
[1] "성긴털/ncn+제비꽃/ncn+은/jxc"      "성긴털/ncn+제비꽃/ncn+은/ncn"     
[3] "성긴털/ncn+제비꽃/ncn+은/jcs"      "성긴털/ncn+제비/ncn+꽃/ncn+은/jxc"
[5] "성긴털/ncn+제비/ncn+꽃/ncn+은/jcs"

$너무
[1] "너무/mag"

$예쁘다
[1] "예쁘/paa+다/ef"

$.
[1] "./sf" "./sy"

## auto-spacing

In [128]:
extractNoun("아버지가방에들어가셨다.")

[1] "아버지" "가방"

In [129]:
extractNoun("아버지가방에들어가셨다.", autoSpacing = T)

[1] "아버지" "방"

In [130]:
MorphAnalyzer("아버지가방에들어가셨다.", autoSpacing = T)

$아버지가
[1] "아버지/ncn+가/jcc"           "아버지/ncn+가/jcs"          
[3] "아버지/ncn+가/ncn"           "아버지/ncn+이/jp+가/ecc"    
[5] "아/xp+버지/ncn+가/jcc"       "아/xp+버지/ncn+가/jcs"      
[7] "아/xp+버지/ncn+가/ncn"       "아/xp+버지/ncn+이/jp+가/ecc"

$방에
[1] "방에/ncn"      "방/nbu+에/jca" "방/nbu+에/jcs" "방/ncn+에/jca"
[5] "방/ncn+에/jcs" "방/xp+에/ncn" 

$들어가
 [1] "들/pvg+어/ecx+가/px+아/ecs"       "들/pvg+어/ecx+가/px+아/ecx"      
 [3] "들/pvg+어/ecx+가/px+아/ef"        "들/pvg+어/ep+가/ecc"             
 [5] "들/xp+어가/ncn"                   "들/xp+어/ncn+가/jcc"             
 [7] "들/xp+어/ncn+가/jcs"              "들/xp+어/ncn+이/jp+가/ecc"       
 [9] "들/pvg+ㄹ/ep+어/ecx+가/px+아/ecs" "들/pvg+ㄹ/ep+어/ecx+가/px+아/ecx"
[11] "들/pvg+ㄹ/ep+어/ecx+가/px+아/ef"  "들/pvg+ㄹ/ep+어/ep+가/ecc"       
[13] "듣/pvg+어/ecx+가/px+아/ecs"       "듣/pvg+어/ecx+가/px+아/ecx"      
[15] "듣/pvg+어/ecx+가/px+아/ef"        "듣/pvg+어/ep+가/ecc"             
[17] "들어가/pvg+아/ecs"                "들어가/pvg+아/ecx"               
[19] "들어가/pvg+아/ef"                

$셨다
[1] "시/paa+었/ep+다/ef"

$.
[1] "./sf" "./sy"

In [131]:
SimplePos09("아버지가방에들어가셨다.", autoSpacing = T)

$아버지가
[1] "아버지/N+가/J"

$방에
[1] "방/N+에/J"

$들어가
[1] "들/P+어/E+가/P+아/E"

$셨다
[1] "시/P+었다/E"

$.
[1] "./S"

In [132]:
SimplePos22("아버지가방에들어가셨다.", autoSpacing = T)

$아버지가
[1] "아버지/NC+가/JC"

$방에
[1] "방/NC+에/JC"

$들어가
[1] "들/PV+어/EC+가/PX+아/EC"

$셨다
[1] "시/PA+었/EP+다/EF"

$.
[1] "./SF"